# AlpesHearth

## Caso de estudio AlpesHearth. 

El jupyter notebook aqui presente se basa en el caso de estudio AlpesHearth, el cual tiene como objetivos:
- Aplicar técnicas de regresión para construir un modelo predictivo que permita estimar el riesgo cardiovascular de una persona siguiendo el ciclo de machine learning.
- Determinar los principales factores de riesgo cardiovascular con base en los datos.
- Aplicar y comprender un modelo de regresión lineal.
- Reconocer posibles sesgos del modelo de aprendizaje de máquina.
- Comunicar de forma clara y sintética los resultados obtenidos.

Se utilizaron las siguientes librerias de Python para el procesamiento y analisis de datos: 
- Pandas
- Scikit-Learn
- Matplotlib, Seaborn

#### Importar las librerias

In [51]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import display

In [52]:
from importlib.metadata import version
print(f"Versión de Pandas: {version('pandas')}")
print(f"Versión de Numpy: {version('numpy')}")
print(f"Versión de Matplotlib: {version('matplotlib')}")
print(f"Versión de Seaborn: {version('seaborn')}")

Versión de Pandas: 2.3.3
Versión de Numpy: 2.2.6
Versión de Matplotlib: 3.10.8
Versión de Seaborn: 0.13.2


### Exploración de los datos

In [53]:
datos_model = pd.read_csv('./Datos/Datos Lab 1.csv')

In [54]:
data = datos_model.copy()

In [55]:
data.head()

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
0,isDx5313,"November 08, 2023",M,44.0,114.300,1.720,38.600,100.000,112/83,228.0,...,High,N,172.000,0.581,112.0,83.0,Hypertension Stage 1,121.0,19.880,HIGH
1,LHCK2961,20/03/2024,F,57.0,92.923,1.842,33.116,106.315,101/91,158.0,...,High,Y,184.172,0.577,101.0,91.0,Hypertension Stage 2,57.0,16.833,INTERMEDIARY
2,WjVn1699,2021-05-27,F,NaN,73.400,1.650,27.000,78.100,90/74,135.0,...,High,N,165.000,0.473,90.0,74.0,Normal,45.0,12.600,LOW
3,dCDO1109,"April 18, 2022",F,35.0,113.300,1.780,35.800,79.600,92/89,158.0,...,Moderate,Y,178.000,0.447,92.0,89.0,Hypertension Stage 1,94.0,14.920,HIGH
4,pnpE1080,01/11/2024,F,48.0,102.200,1.750,33.400,106.700,121/68,207.0,...,Low,Y,175.000,0.610,121.0,68.0,Elevated,128.0,18.870,HIGH


Aqui podemos ver como hay columnas de datos repetidos y/o muy parecidos. Por ejemplo, Height (m) y Height (cm), Waist-to-Height Ratio Height y Abdominal Circumference, BMI Weight y Height etc.

A la hora de realizar el analisis estadistico, sera necesario ver cuales de estas variables son redundantes y no nos aportan al modelo. Asimismo, hay que revisar para casos tales como las dos estaturas, si coinciden. De no coincidir, habra que realizar un promedio o utilizar alguna otra tecnica.

In [56]:
display(data.sample(5))

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
323,LiFG5942,05/09/2025,M,51.0,75.800,1.900,21.000,96.800,142/66,280.0,...,Low,Y,190.000,0.509,142.0,66.0,Hypertension Stage 2,198.0,18.900,HIGH
870,QtBs0839,24 Aug 25,M,41.0,112.716,1.770,38.307,74.330,153/95,259.0,...,Moderate,Y,177.027,0.420,153.0,95.0,Hypertension Stage 2,175.0,20.491,LOW
789,CpuL1643,"January 25, 2020",F,38.0,112.400,1.690,39.400,101.200,139/77,209.0,...,High,N,169.000,0.599,139.0,77.0,Hypertension Stage 1,139.0,19.010,INTERMEDIARY
594,qZYa2730,26 Nov 20,M,56.0,80.567,1.597,22.383,86.291,108/101,249.0,...,Low,Y,159.723,0.540,108.0,101.0,Hypertension Stage 2,156.0,14.857,INTERMEDIARY
684,qoOw6598,16 Apr 23,F,60.0,116.200,NaN,36.700,97.800,112/80,188.0,...,Moderate,N,NaN,0.549,112.0,80.0,Hypertension Stage 1,106.0,16.700,INTERMEDIARY


In [57]:
data.shape

(1639, 24)

Esto nos dice que el conjunto de datos esta compuesto por 1.639 registros (filas) y 24 variables (columnas). 

In [58]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient ID                    1639 non-null   object 
 1   Date of Service               1639 non-null   object 
 2   Sex                           1639 non-null   object 
 3   Age                           1571 non-null   float64
 4   Weight (kg)                   1566 non-null   float64
 5   Height (m)                    1578 non-null   float64
 6   BMI                           1586 non-null   float64
 7   Abdominal Circumference (cm)  1578 non-null   float64
 8   Blood Pressure (mmHg)         1639 non-null   object 
 9   Total Cholesterol (mg/dL)     1571 non-null   float64
 10  HDL (mg/dL)                   1557 non-null   float64
 11  Fasting Blood Sugar (mg/dL)   1585 non-null   float64
 12  Smoking Status                1639 non-null   object 
 13  Dia

Arriba podemos ver el detalle de cada columna, como el nombre, el número de valores no nulos (en este caso al tener 1.639, indica que no se tienen valores faltantes) y el tipo de los datos de cada columna, al igual que el número de columnas con cada tipo.

Es interesante ver que las columnas que no son numéricas (int o float) son `Patient ID`, `Date of Service`, `Sex`, `Blood Pressure (mmHg)`, `Smoking Status`, `Diabetes Status`, `Physical Activity Level`, `Family History of CVD`, y `Blood Pressure Category`, las cuales están en formato object, esto es algo importante y que debemos tener en cuenta. 

Como sabemos que `Blood Pressure Category` = `Systolic BP` / `Diastolic BP`, habra que reemplazar los NULLs por los respectivos valores de `Blood Pressure Category`. Mas adelante se revisara si estos valores son redundantes y si sera necesario remover alguna de estas columnas.

Un insumo imprescindible para entender un conjunto de datos es el diccionario, ya que este nos permite conocer el significado de cada variable y sus rangos válidos.  

Al mismo tiempo, al revisar los valores non-null de cada una de las columnas, podemos ver que hay muchos datos NULL dentro de la base de datos. Por lo que se debera realizar algo al respecto.

In [59]:
cols_con_null = []

for col in data.columns:
    if data[col].count() != len(data):
        cols_con_null.append(col)

cols_con_null

['Age',
 'Weight (kg)',
 'Height (m)',
 'BMI',
 'Abdominal Circumference (cm)',
 'Total Cholesterol (mg/dL)',
 'HDL (mg/dL)',
 'Fasting Blood Sugar (mg/dL)',
 'Height (cm)',
 'Waist-to-Height Ratio',
 'Systolic BP',
 'Diastolic BP',
 'Estimated LDL (mg/dL)',
 'CVD Risk Score']

In [60]:
diccionario = pd.read_excel('./Datos/DiccPacientes.xlsx')
pd.set_option('display.max_colwidth', None)
diccionario

,Nombre Columna,Tipo de dato,Comentarios
0,Patient ID,String,Identificador del paciente
1,Date of Service,Date,Fecha de la atención
2,Sex,String,"Sexo (Femenino, Masculino)"
3,Age,Integer,Edad
4,Weight (kg),Float,Peso
5,Height (m),Float,Altura
6,BMI,Float,Índice de masa corporal
7,Abdominal Circumference (cm),Float,Circunferencia abdominal
8,Blood Pressure (mmHg),String,"Presión sanguínea, de la forma ""<Presión arterial sistólica>/<Presión arterial diastólica>"""
9,Total Cholesterol (mg/dL),Float,Colesterol total


Con el fin de enriquecer el modelo, se revisaran las siguientes variables mas adelante:

Derivadas de datos existentes
- Grupo etario
- Categoria de BMI
- Categoria de glucosa
- Ratio colesterol
- Pulse Pressure
- Mean Arterial Pressure

Fuentes externas
- Mes
- Dias de la semana
- Estacion del anio

Scores clinicos establecidos
- Framingham Risk Score
- ASCVD Risk Score

Interacciones
- Fumador + Diabetes
- BMI * Actividad Fisica
- Edad * Presion Arterial

In [61]:
data.describe()

,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Total Cholesterol (mg/dL),HDL (mg/dL),Fasting Blood Sugar (mg/dL),Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Estimated LDL (mg/dL),CVD Risk Score
count,1571.000000,1566.000000,1578.000000,1586.000000,1578.000000,1571.000000,1557.000000,1585.000000,1571.000000,1563.000000,1578.000000,1554.000000,1582.000000,1610.000000
mean,46.803186,85.666006,1.757439,28.424744,91.538861,199.043673,56.183558,117.836860,175.770082,0.522440,125.632637,82.887536,113.235896,18.227281
std,13.039479,21.712504,0.118012,7.309275,13.427985,59.388670,16.721702,32.379634,11.695880,0.085692,22.577463,15.503625,61.435291,10.767666
min,6.134000,13.261000,1.371000,4.317000,49.542000,-1.256000,0.008000,15.306000,136.498000,0.250000,49.914000,31.720000,-92.055000,-20.057000
25%,37.000000,67.100000,1.666500,22.600000,79.700000,150.000000,42.000000,92.000000,167.000000,0.453000,108.000000,71.000000,62.000000,15.150000
50%,46.000000,86.314000,1.760000,28.000000,91.200000,199.000000,56.000000,115.000000,176.000000,0.519000,125.000000,82.000000,112.000000,16.967000
75%,55.000000,104.801500,1.850000,33.963000,102.267250,250.000000,70.000000,139.000000,185.000000,0.582000,141.000000,93.000000,159.000000,18.900000
max,89.420000,158.523000,2.146000,53.028000,136.336000,385.679000,110.315000,219.667000,214.394000,0.804000,202.711000,134.066000,317.314000,114.980000


Para el peso seria prudente revisar si es posible? Es decir si se podria un individuo de 13kg con 20 anios? 

Un valor de 0.085 de Waist to Height es extremadamente inusual y parece ser inventado. Es posible que no sea un valor correcto. Posiblemente un valor un poco inferior a 0.35 (aunque ya muy bajo) pordia ser posible. 

Un Systolic BP de 22.577 es sumamente improbable y muy posiblemente un error de medicion. 

Un Estimated LDL de -92.05 es fisiologicamente imposible. Por lo que es posible que este valor no sea real. 

Un valor negativo de CVD Risk Score no es posible. Los valores posibles son entre 0 y 100. Por lo que hay errores en los minimos y maximos.

Aqui podemos realizar varias tecnicas de normalizacion con el fin de mejorar la habilidad predictiva del modelo.

In [62]:
# Revisar formatos, asegurarse que no hayan representaciones diferentes de los mismos datos...

objetos = ['Patient ID', 'Date of Service', 'Sex', 
           'Blood Pressure (mmHg)', 'Smoking Status', 
           'Diabetes Status', 'Physical Activity Level', 
           'Family History of CVD', 'Blood Pressure Category', 
           'CVD Risk Level'
] 

for i in objetos:
    display(data[i].value_counts())

Patient ID
AhYt1346    3
RlsB8509    3
RwGu5647    3
dJuC5084    3
YLCe2926    3
           ..
Srzz2840    1
nMHG7307    1
VpjT3887    1
CNBz8190    1
qgQI1151    1
Name: count, Length: 1376, dtype: int64

Date of Service
09-20-2023           6
December 05, 2025    5
December 02, 2020    4
01/12/2024           4
08-09-2022           4
                    ..
2025-02-16           1
April 14, 2021       1
07/09/2021           1
12-13-2024           1
22/10/2021           1
Name: count, Length: 1274, dtype: int64

Sex
M    821
F    818
Name: count, dtype: int64

Blood Pressure (mmHg)
124/72    6
127/84    6
129/61    5
121/68    5
142/66    4
         ..
136/77    1
142/96    1
149/68    1
95/81     1
141/96    1
Name: count, Length: 1152, dtype: int64

Smoking Status
Y    850
N    789
Name: count, dtype: int64

Diabetes Status
N    821
Y    818
Name: count, dtype: int64

Physical Activity Level
High        582
Moderate    537
Low         520
Name: count, dtype: int64

Family History of CVD
N    820
Y    819
Name: count, dtype: int64

Blood Pressure Category
Hypertension Stage 2    680
Hypertension Stage 1    527
Normal                  321
Elevated                111
Name: count, dtype: int64

CVD Risk Level
HIGH            793
INTERMEDIARY    616
LOW             230
Name: count, dtype: int64

Aqui podemos ver tanto que hay IDs repetidos, como que hay formatos diferentes para las fechas de servicio, por lo que sera necesario revisar si es que hay filas de datos duplicadas, y refactorizar las fechas a un formato estandar. 

In [63]:
display(data.duplicated().sum())

display(data[data.duplicated(keep=False)])

np.int64(151)

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
10,oSqq7069,18 Jun 23,F,38.0,68.516,1.929,39.967,96.957,105/84,241.000,...,Moderate,Y,192.936,0.503,105.0,84.0,Hypertension Stage 1,159.0,18.063,HIGH
14,gBFe4249,04/02/2020,F,52.0,89.700,1.880,25.400,107.500,145/92,142.000,...,Low,Y,188.000,0.572,145.0,92.0,Hypertension Stage 2,42.0,17.170,HIGH
17,AhYt1346,09-28-2020,M,41.0,71.300,1.730,23.800,107.900,139/61,253.000,...,Low,Y,173.000,NaN,139.0,NaN,Hypertension Stage 1,146.0,16.770,HIGH
29,RlsB8509,29/08/2020,F,55.0,79.700,1.780,25.200,86.900,140/62,248.000,...,Moderate,Y,178.000,0.488,140.0,62.0,Hypertension Stage 2,182.0,19.000,HIGH
31,RwGu5647,"May 03, 2020",M,42.0,51.095,1.521,20.190,72.274,159/91,296.000,...,Moderate,N,152.080,0.259,159.0,91.0,Hypertension Stage 2,211.0,17.908,LOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,KiRC8867,20/12/2024,M,46.0,90.748,1.705,30.866,108.768,105/113,198.000,...,Moderate,N,170.518,0.638,105.0,113.0,Hypertension Stage 2,117.0,15.383,INTERMEDIARY
1614,XUjh9519,2022-08-15,F,55.0,73.500,1.880,20.800,82.700,144/72,21.662,...,Low,Y,188.000,0.440,144.0,72.0,Hypertension Stage 2,100.0,16.680,HIGH
1624,rZFB5714,05 Sep 21,M,NaN,55.600,1.830,16.600,104.800,126/95,289.000,...,High,N,183.000,0.573,126.0,95.0,Hypertension Stage 2,201.0,15.400,INTERMEDIARY
1633,gBFe4249,04/02/2020,F,52.0,89.700,1.880,25.400,107.500,145/92,142.000,...,Low,Y,188.000,0.572,145.0,92.0,Hypertension Stage 2,42.0,17.170,HIGH


Aqui vemos las filas de datos que se encuentran completamente duplicadas. Estas se removeran en la siguiente fase del proyecto.

### Preparación de datos

In [64]:
data = data.drop_duplicates()

Aqui se quitan las filas de datos duplicados.

In [65]:
data = data.copy()
data["Date of Service"] = pd.to_datetime(data["Date of Service"], format='mixed')
data
# TODO: Como se usa formato 'mixed', para datos como: 08-09-2022, se adivina cual es el mes y cual es el dia.

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
0,isDx5313,2023-11-08,M,44.0,114.300,1.720,38.600,100.000,112/83,228.0,...,High,N,172.000,0.581,112.0,83.0,Hypertension Stage 1,121.0,19.880,HIGH
1,LHCK2961,2024-03-20,F,57.0,92.923,1.842,33.116,106.315,101/91,158.0,...,High,Y,184.172,0.577,101.0,91.0,Hypertension Stage 2,57.0,16.833,INTERMEDIARY
2,WjVn1699,2021-05-27,F,NaN,73.400,1.650,27.000,78.100,90/74,135.0,...,High,N,165.000,0.473,90.0,74.0,Normal,45.0,12.600,LOW
3,dCDO1109,2022-04-18,F,35.0,113.300,1.780,35.800,79.600,92/89,158.0,...,Moderate,Y,178.000,0.447,92.0,89.0,Hypertension Stage 1,94.0,14.920,HIGH
4,pnpE1080,2024-01-11,F,48.0,102.200,1.750,33.400,106.700,121/68,207.0,...,Low,Y,175.000,0.610,121.0,68.0,Elevated,128.0,18.870,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1632,ioby2183,2024-08-13,M,40.0,120.000,NaN,35.400,100.900,94/68,223.0,...,Moderate,Y,184.000,0.548,94.0,68.0,Normal,131.0,18.240,HIGH
1634,mrzf5858,2021-05-21,F,35.0,77.600,1.780,24.500,84.600,124/90,143.0,...,Low,N,178.000,0.475,124.0,90.0,Hypertension Stage 2,37.0,13.960,LOW
1635,nPnN5477,2022-12-04,F,35.0,92.005,1.726,NaN,98.692,95/111,156.0,...,High,N,172.602,0.572,95.0,111.0,Hypertension Stage 2,46.0,14.316,LOW
1637,QSFT6794,2025-09-06,M,49.0,NaN,1.630,23.100,93.800,144/91,191.0,...,Moderate,Y,163.000,0.575,144.0,NaN,Hypertension Stage 2,82.0,17.640,HIGH


Aqui se realiza la respectiva refactorizacion de las fechas.

In [ ]:
# TODO: Revisar!!
# Convertir valores imposibles a NaN para luego imputar

# Weight: mínimo ~30kg para adulto, ~10kg para niño
data.loc[data['Weight (kg)'] < 20, 'Weight (kg)'] = np.nan

# Waist-to-Height Ratio: rango realista 0.3 - 0.9
data.loc[data['Waist-to-Height Ratio'] < 0.3, 'Waist-to-Height Ratio'] = np.nan
data.loc[data['Waist-to-Height Ratio'] > 0.9, 'Waist-to-Height Ratio'] = np.nan

# Systolic BP: rango realista 70 - 250
data.loc[data['Systolic BP'] < 70, 'Systolic BP'] = np.nan
data.loc[data['Systolic BP'] > 250, 'Systolic BP'] = np.nan

# Diastolic BP: rango realista 40 - 150
data.loc[data['Diastolic BP'] < 40, 'Diastolic BP'] = np.nan
data.loc[data['Diastolic BP'] > 150, 'Diastolic BP'] = np.nan

# Estimated LDL: no puede ser negativo, rango 20 - 300
data.loc[data['Estimated LDL (mg/dL)'] < 0, 'Estimated LDL (mg/dL)'] = np.nan
data.loc[data['Estimated LDL (mg/dL)'] > 300, 'Estimated LDL (mg/dL)'] = np.nan

# CVD Risk Score: rango 0 - 100
data.loc[data['CVD Risk Score'] < 0, 'CVD Risk Score'] = np.nan
data.loc[data['CVD Risk Score'] > 100, 'CVD Risk Score'] = np.nan

# Total Cholesterol: no puede ser negativo
data.loc[data['Total Cholesterol (mg/dL)'] < 0, 'Total Cholesterol (mg/dL)'] = np.nan

# HDL: rango realista 10 - 150
data.loc[data['HDL (mg/dL)'] < 10, 'HDL (mg/dL)'] = np.nan

In [66]:
# Conversion a m
height_m = data["Height (m)"]
height_cm_to_m = data["Height (cm)"] / 100

# Promedio donde ambos existen, sino el que existe
data.loc[:,"Height (m)"] = (height_m.fillna(height_cm_to_m) + height_cm_to_m.fillna(height_m)) / 2

data = data.drop(columns=["Height (cm)"])

Para el parametro height, se realizo: 
- Si existian tanto Height (m), como Height (cm), se calculo un promedio.
- Si solo existia uno de estos se dejo este.
- Si no existia se tomo como el promedio global de las estaturas.

In [67]:
BMI = (data["Weight (kg)"] / data["Height (m)"] ** 2)
data["BMI"] = data["BMI"].fillna(BMI)

Para los datos que no tenian BMI, se utilizo la formula del BMI para agregarla.
BMI = weight (kg) / height (m) ^2

In [68]:
bp_split = data['Blood Pressure (mmHg)'].str.split('/', expand=True).astype(float)
data['Systolic BP'] = data['Systolic BP'].fillna(bp_split[0])
data['Diastolic BP'] = data['Diastolic BP'].fillna(bp_split[1])

Aqui llenamos los datos restantes de `Systolic BP` y `Diastolic BP` con los respectivos datos que tienen en `Blood Pressure (mmHg)`.

In [69]:
WHtR = data['Abdominal Circumference (cm)'] / (data['Height (m)'] * 100)
data['Waist-to-Height Ratio'] = data['Waist-to-Height Ratio'].fillna(WHtR)

# Verificar rangos
print(data['Waist-to-Height Ratio'].describe())

count    1487.000000
mean        0.523331
std         0.085292
min         0.250000
25%         0.455000
50%         0.519000
75%         0.582000
max         0.825281
Name: Waist-to-Height Ratio, dtype: float64


Si bien es cierto que waist circumference es diferente de abdominal circumference, sigue siendo una aproximacion valida y suelen estar linealmente correlacionadas. Por lo que, para las filas de datos en las que faltan valores para el waist to height ratio, se decidio utilizar el rubro `Abdominal Circumference (cm)` y dividirlo entre el rubro de `Height (m)` * 100.

In [71]:
abdominal_circum = data['Height (m)'] * data['Waist-to-Height Ratio']/100
data['Abdominal Circumference (cm)'] = data['Abdominal Circumference (cm)'].fillna(abdominal_circum)

weight = data['BMI'] * (data['Height (m)'] ** 2)
data['Weight (kg)'] = data['Weight (kg)'].fillna(weight)

estimated_LDL = data['Total Cholesterol (mg/dL)'] - data['HDL (mg/dL)'] - 30
data['Estimated LDL (mg/dL)'] = data['Estimated LDL (mg/dL)'].fillna(estimated_LDL)

tc_calc = data['Estimated LDL (mg/dL)'] + data['HDL (mg/dL)'] + 30
data['Total Cholesterol (mg/dL)'] = data['Total Cholesterol (mg/dL)'].fillna(tc_calc)

hdl_calc = data['Total Cholesterol (mg/dL)'] - data['Estimated LDL (mg/dL)'] - 30
data['HDL (mg/dL)'] = data['HDL (mg/dL)'].fillna(hdl_calc)

Aqui, utilizamos los datos que ya tenemos para calcular algunos de los que faltan. Esto se pudo realizar para los rubros: 
- `Abdominal Circumference (cm)`
- `Weight (kg)`
- `Estimated LDL (mg/dL)`
- `Total Cholesterol (mg/dL)`
- `HDL (mg/dL)`

Sin embargo, siguen faltando algunos datos. 

In [72]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 1488 entries, 0 to 1638
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Patient ID                    1488 non-null   object        
 1   Date of Service               1488 non-null   datetime64[ns]
 2   Sex                           1488 non-null   object        
 3   Age                           1427 non-null   float64       
 4   Weight (kg)                   1487 non-null   float64       
 5   Height (m)                    1485 non-null   float64       
 6   BMI                           1487 non-null   float64       
 7   Abdominal Circumference (cm)  1487 non-null   float64       
 8   Blood Pressure (mmHg)         1488 non-null   object        
 9   Total Cholesterol (mg/dL)     1482 non-null   float64       
 10  HDL (mg/dL)                   1483 non-null   float64       
 11  Fasting Blood Sugar (mg/dL)   1439 

In [ ]:
datos_prueba = pd.read_csv(
    './Datos/Datos Test Lab 1.csv',
    sep=';'
)